# 

# Introduction

This is information The Johns Hopkins University Hospital trying to do a prediction of the chances of one being Sepssis Positive or Negative

## Null Hypothesis:

There is no significant difference in the mean age between patients with positive and negative "Sepssis" conditions.

## Alternate Hypothesis:

There is a significant difference in the mean age between patients with positive and negative "Sepssis" conditions.

## Questions

1. Between Positive and Negative Sepssis which one has a higher numbers ? 
2. Is there a correlation between a Sepssis and Age Bracket ?
3. What is the Insurance uptake like for those with Positive Sepssis compared to Negative Sepssis ?
4. What is the Relationship between Blood Pressure, Body mass index and Age of the Patients ?

In [1]:
# importing libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
import seaborn as sns

#Data Splitting
from sklearn.model_selection import train_test_split, GridSearchCV

#Models
from sklearn.metrics import confusion_matrix,classification_report,f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy import stats
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder,StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score
import pickle
import shap

# EDA
from pandas_profiling import ProfileReport

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Data handling
#import pandas as pd

# Vizualisation (Matplotlib, Plotly, Seaborn, etc. )
...

# EDA (pandas-profiling, etc. )
...

# Feature Processing (Scikit-learn processing, etc. )
...

# Machine Learning (Scikit-learn Estimators, Catboost, LightGBM, etc. )
...

# Hyperparameters Fine-tuning (Scikit-learn hp search, cross-validation, etc. )
...

# Other packages
import os


## Data Loading

In [ ]:
# loading dataset
patient_test= pd.read_csv(r"C:\Users\hp\Box\Azubi-Africa\LP6\Sepsis-Prediction-1\Data Files\Paitients_Files_Test.csv")
patient_train= pd.read_csv(r"C:\Users\hp\Box\Azubi-Africa\LP6\Sepsis-Prediction-1\Data Files\Paitients_Files_Train.csv")

# Exploratory Data Analysis: EDA

## Data Overview

In [ ]:
patient_test.head()

In [ ]:
patient_train.head()

In [ ]:
patient_train.shape

In [ ]:
patient_train.info()

In [ ]:
patient_train.isna().sum()

In [ ]:
patient_train.duplicated().sum()

In [ ]:
patient_train.describe()

# Exploratory Data Analysis

### Between Positive and Negative Sepssis which one has a higher numbers ? 

In [ ]:
# Getting the value count of Sepssis(positive/negative)
patient_train['Sepssis'].value_counts()

In [ ]:
sns.countplot(x='Sepssis', data=patient_train)
plt.title('Sepssis Value Counts')
plt.xlabel('Sepssis')
plt.ylabel('Count')
plt.show()

From the above graph we are able to note that we have more Negative results as compared to Positive

### Is there a correlation between a Sepssis and Age Bracket ?

In [ ]:
# Define age groups
age_groups = ["18-30", "31-40", "41-50", "51-60", "61-70","71-80"]

# Count occurrences of Sepssis for each age group
grouped_data = {age_group: {"Positive": 0, "Negative": 0} for age_group in age_groups}
for age, sepssis in zip(patient_train["Age"], patient_train["Sepssis"]):
    for age_range in age_groups:
        age_range_values = age_range.split("-")
        if int(age_range_values[0]) <= age <= int(age_range_values[1]):
            grouped_data[age_range][sepssis] += 1

# Get the counts of Negative and Positive Sepssis for each age group
negative_counts = [grouped_data[age_group]["Negative"] for age_group in age_groups]
positive_counts = [grouped_data[age_group]["Positive"] for age_group in age_groups]

# Create a bar plot to compare Negative and Positive Sepssis
bar_width = 0.35
index = range(len(age_groups))

plt.bar(index, negative_counts, bar_width, label='Negative Sepssis')
plt.bar(index, positive_counts, bar_width, bottom=negative_counts, label='Positive Sepssis')

# Set x-axis ticks and labels
plt.xlabel('Age Groups')
plt.xticks(index, age_groups)

# Set y-axis label
plt.ylabel('Count')

# Set title and legend
plt.title('Comparison of Negative and Positive Sepssis by Age Group')
plt.legend()

# Display the plot
plt.show()

We are able to conclude there is a higher percantage of positive and negative sepssis in the age bracket of 18-30 yrs

### What is the Relationship between Blood Pressure, Body mass index and Age of the Patients ?

In [ ]:
# Set the figure size
plt.figure(figsize=(15, 8))  # Adjust the width and height as desired

# Create the scatter plot
plt.scatter(patient_train['PR'], patient_train['M11'], c=patient_train['Age'], cmap='viridis')
plt.colorbar(label='Age')
plt.xlabel('PR')
plt.ylabel('M11')
plt.title('Relationship between PR, M11, and Age')

# Display the scatter plot
plt.show()

### What is the Insurance uptake like for those with Positive Sepssis compared to Negative Sepssis

In [ ]:
# Count the occurrences of each combination of Sepssis and Insurance
count_data = patient_train.groupby(['Sepssis', 'Insurance']).size().reset_index(name='Count')

# Create a bar plot
ax = sns.barplot(x='Sepssis', y='Count', hue='Insurance', data=count_data)

# Add value annotations to the plot
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'), (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

# Set labels and title
plt.xlabel('Sepssis')
plt.ylabel('Count')
plt.title('Distribution of Sepssis Grouped by Insurance')

# Display the plot
plt.show()

We can conclude that insurance uptake is lower for those with positive sepssis as compared to those who are negative

In [ ]:
# Select the columns for individual box plots
columns_for_boxplot = ['PRG', 'PL', 'PR', 'SK', 'TS', 'M11', 'BD2', 'Age']

# Create individual box plots
plt.figure(figsize=(20, 15))

for i, col in enumerate(columns_for_boxplot):
    plt.subplot(3, 3, i+1)
    sns.boxplot(data=patient_train[col])
    plt.xlabel(col)
    plt.ylabel('Values')

plt.tight_layout()
plt.show()

We are able to note that there are outliners across

## Multivariate Analysis

In [ ]:
# Calculate the correlation matrix
corr_matrix = patient_train.drop(columns=['ID']).corr()

# Generate the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='gist_heat', square=True)

# Set the title of the heatmap
plt.title('Correlation Heatmap')

# Display the heatmap
plt.show()

#### Based on the heatmap, we can make the following observations:
-> Age and PRG have a moderately positive correlation, indicating that as the number of pregnancies increases, the age tends to be higher.

-> PL and PR have a weak positive correlation, suggesting that higher glucose levels may be associated with slightly higher diastolic blood pressure.

-> M11 and Age have a weak negative correlation, implying that older individuals tend to have slightly lower values for M11.

-> TS and PRG have a weak negative correlation, indicating that as the number of pregnancies increases, the triceps skinfold thickness tends to be slightly lower.

-> There seems to be no significant correlation between other variables based on the given dataset.

In [ ]:
numerical_vars = ['PRG', 'PL', 'PR']
sns.pairplot(data=patient_train, vars=numerical_vars, hue='Sepssis', height=4)

In [ ]:
numerical_vars = ['SK', 'TS', 'M11']
sns.pairplot(data=patient_train, vars=numerical_vars, hue='Sepssis', height=6)

In [ ]:
numerical_vars = ['BD2', 'Age', 'Insurance']
sns.pairplot(data=patient_train, vars=numerical_vars, hue='Sepssis', height=6)

In [ ]:
plt.figure(figsize=(10, 8))  # Set the figure size
plt.hist(patient_train['Age'], bins=10)
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Age Distribution')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))  # Set the figure size

sns.boxplot(x='Sepssis', y='PR', data=patient_train)
plt.xlabel('Sepssis')
plt.ylabel('Blood Pressure (mm Hg)')
plt.title('Blood Pressure Distribution by Sepsis')
plt.show()

## Testing Null Hypothesis
### There is no significant difference in the mean age between patients with positive and negative "Sepssis" conditions.

In [ ]:
# Separate the age values for positive and negative Sepssis groups
age_positive = patient_train[patient_train['Sepssis'] == 'Positive']['Age']
age_negative = patient_train[patient_train['Sepssis'] == 'Negative']['Age']

# Perform the two-sample t-test
t_statistic, p_value = stats.ttest_ind(age_positive, age_negative)

# Define the significance level
alpha = 0.05

# Compare the p-value with the significance level to draw conclusions
if p_value < alpha:
    print("Reject the null hypothesis. There is a significant difference in the mean age.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference in the mean age.")

## Testing Alternate Hypothesis
### There is a significant difference in the mean age between patients with positive and negative "Sepssis" conditions.

In [ ]:
# Separate the age values for patients with positive and negative Sepssis
positive_sepssis = patient_train[patient_train['Sepssis'] == 'Positive']['Age']
negative_sepssis = patient_train[patient_train['Sepssis'] == 'Negative']['Age']

# Perform an independent samples t-test
t_statistic, p_value = stats.ttest_ind(positive_sepssis, negative_sepssis)

# Print the results
print("T-statistic:", t_statistic)
print("P-value:", p_value)

We can interpret the results as follows:

T-Statistic: The t-statistic measures the difference between the mean ages of the positive and negative Sepssis groups relative to the variation within each group. In this case, the t-statistic is 5.254202967191447, indicating a significant difference between the mean ages of the two groups.

P-Value: The p-value represents the probability of obtaining the observed difference in means or a more extreme difference if the null hypothesis (no significant difference) were true. In this case, the p-value is very small (2.0718778891882012e-07), which is significantly lower than the conventional significance level of 0.05. Therefore, we reject the null hypothesis and conclude that there is a significant difference in the mean age between patients with positive and negative Sepssis conditions.

In summary, based on the results, we can say that there is strong evidence to support the claim that there is a significant difference in the mean age between patients with positive and negative Sepssis conditions.

# Feature Processing & Engineering

## Drop Duplicates

In [ ]:
# Check for duplicate rows
duplicate_rows = patient_train.duplicated()
print("Number of duplicate rows:", duplicate_rows.sum())

## Feature Encoding

In [ ]:
# spliting the data into feature and target variables
X = patient_train.iloc[:,:-1]
y = patient_train.iloc[:,-1]

In [ ]:
# Encoding the target variable
label_encoder=LabelEncoder()
y_encoded = label_encoder.fit_transform(y) 

In [ ]:
#converting to a DataFrame
y_encoded_df = pd.DataFrame(y_encoded, columns = ["Sepssis"])

In [ ]:
# combining the features and the encoded target variables
patient_df = pd.concat([X, y_encoded_df], axis = 1 )

In [ ]:
patient_df.head()

In [ ]:
patient_df.drop('ID',axis=1,inplace=True)

## Data Splitting

In [ ]:
# Split the encoded data into train and test sets
X_train, X_eval, y_train, y_eval = train_test_split(patient_df.iloc[:, :-1], patient_df.iloc[:, -1:],
                                                    test_size=0.2, random_state=42, stratify=patient_df.iloc[:, -1:])

In [ ]:
X_train.shape, X_eval.shape, y_train.shape, y_eval.shape

In [ ]:
X_test = patient_test.drop('ID', axis=1)

## Imputing Missing Values

In [ ]:
# Creating imputer variables
numerical_imputer = SimpleImputer(strategy = "mean")

In [ ]:
X_train_imputed = numerical_imputer.fit_transform(X_train)
X_eval_imputed = numerical_imputer.transform(X_eval)

## Features Scaling

In [ ]:
#Create a Scaler and fit it to your training data
scaler = StandardScaler()
scaler.fit(X_train_imputed)

# Transform the data
scaled_df = scaler.transform(X_train_imputed)
X_train_df = pd.DataFrame(scaled_df, columns=['PRG','PL','PR','SK','TS','M11','BD2','Age','Insurance'])

X_eval_scaled = scaler.transform(X_eval_imputed)
X_eval_df = pd.DataFrame(X_eval_scaled, columns = ['PRG','PL','PR','SK','TS','M11','BD2','Age','Insurance'])

In [ ]:
X_train_df

## Checking for Class Imbalance

In [ ]:
counts = patient_train['Sepssis'].value_counts()

# Calculate the percentage of each class in the 'Sepssis' column
percentages = counts / counts.sum() * 100

# Print the results
print(percentages)

In [ ]:
# Define the data for the chart
labels = percentages.index.tolist()
sizes = percentages.values.tolist()

# Create the chart
fig, ax = plt.subplots(figsize = (8, 6))
ax.pie(sizes, labels=labels, autopct='%1.1f%%')
ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Class Imbalance in Sepssis (Percentage)')
plt.legend()

# Show the chart
plt.show()

# Machine Learning Modeling

## Decision Tree Regression Model

In [ ]:
#fitting decision tree model
dt_model=DecisionTreeClassifier(random_state=42)

In [ ]:
#fitting model on imbalanced data
dt_model.fit(X_train_df,y_train)


In [ ]:
dt_pred = dt_model.predict(X_eval_df)

In [ ]:
# report on unbalanced data
dt_report1 = classification_report(y_eval, dt_pred)
print(dt_report1)

## Random Forest Model

In [ ]:
#fitting random forest model
rf_model = RandomForestClassifier(random_state=42)

In [ ]:
#fitting model on imbalanced data
rf_model.fit(X_train_df,y_train)

In [ ]:
rf_pred = rf_model.predict(X_eval_df)

In [ ]:
# report on unbalanced data
rf_report1 = classification_report(y_eval, rf_pred)
print(rf_report1)

## Logistic Regression Model

In [ ]:
# Fit a logistic regression model to the training data
lr_model = LogisticRegression(random_state=42)

In [ ]:
#fitting model on imbalanced data
lr_model.fit(X_train_df,y_train)

In [ ]:
lr_pred = lr_model.predict(X_eval_df)

In [ ]:
# report on unbalanced data
lr_report1 = classification_report(y_eval, lr_pred)
print(lr_report1)

## K Nearest Neighbour Model

In [ ]:
kn_model = KNeighborsClassifier(n_neighbors=5)

In [ ]:
#fitting model on imbalanced data
kn_model.fit(X_train_df,y_train)

In [ ]:
kn_pred = kn_model.predict(X_eval_df)

In [ ]:
# report on unbalanced data
kn_report1 = classification_report(y_eval, kn_pred)
print(kn_report1)

Comparing the models, we can make the following observations:

-> The logistic regression model has the highest precision, recall, and F1-score for both classes, indicating better performance in correctly predicting both positive and negative instances.

-> The random forest model has the lowest precision, recall, and F1-score for both classes among the compared models.

-> The decision tree regression model and the K Nearest Neighbour model have similar performance metrics, with moderate precision, recall, and F1-scores for both classes.

Overall, the Logistic Regression model appears to be the best-performing model among the compared models based on the provided evaluation metrics. 

## Model Comparison

The F1-score is a measure of the accuracy of a binary classifier. It is calculated by taking the harmonic mean of precision and recall. Precision is the fraction of predicted positives that are actually positive, and recall is the fraction of actual positives that are predicted positive.

In [ ]:
dt_f1= f1_score(y_eval, dt_pred)
rf_f1= f1_score(y_eval, rf_pred)
kn_f1= f1_score(y_eval, kn_pred)
lr_f1= f1_score(y_eval, lr_pred)

In [ ]:
results= {'Model Name':['DecisionTreeClassifier','RandomForestClassifier','KNN','LogisticRegression'],
         'f1_score':[dt_f1,rf_f1,kn_f1,lr_f1]}
results_df= pd.DataFrame(results)

In [ ]:
results_df

The results show that the Logistic Regression model has the highest F1 score, followed by the DecisionTree Classifier model, the RandomForest Classifier model, and the KNN model. This means that the Logistic Regression model is the best model for predicting the target variable. The higher the F1 score, the better the model is at predicting the target variable. However, all of the models are doing a good job of predicting whether or not a patient has sepsis.

## Hyperparameter Tuning

In [ ]:
# Define the parameter grid for the decision tree classifier
dt_param = {
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [5,10 ,15]
}


# perform a grid search with 5-fold cross-validation using only the selected features
dt_grid_search = GridSearchCV(estimator=dt_model, param_grid=dt_param,scoring='f1',cv=5)
dt_grid_search.fit(X_train_df,y_train)

# print the best hyperparameters and the corresponding mean cross-validation score
print("Best hyperparameters: ", dt_grid_search.best_params_)
print('Best estimators: ',dt_grid_search.best_estimator_)
print("Best f1_score: ", dt_grid_search.best_score_)

In [ ]:
# define the hyperparameter grid to search over the logistics regression model
lr_param = {
    'C': [200,300,400,500],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

# perform a grid search with 5-fold cross-validation using only the selected features
lr_grid_search = GridSearchCV(estimator=lr_model, param_grid=lr_param,scoring='f1',cv=5)
lr_grid_search.fit(X_train_df, y_train)

# print the best hyperparameters and the corresponding mean cross-validation score
print("Best hyperparameters: ", lr_grid_search.best_params_)
print('Best estimators: ',lr_grid_search.best_estimator_)
print("Best f1_score average: ", lr_grid_search.best_score_)

In [ ]:
# define the parameter grid for knn model
kn_param = {'n_neighbors': [20,25,30,35,40],
              'weights': ['uniform', 'distance'],
              'p': [1, 2]}
# perform grid search with cross-validation
kn_grid_search = GridSearchCV(kn_model, param_grid=kn_param,scoring='f1', cv=5)
kn_grid_search.fit(X_train_df, y_train)

# print the best hyperparameters and the corresponding score
print("Best hyperparameters: ",kn_grid_search.best_params_)
print('Best estimators: ',kn_grid_search.best_estimator_)
print("Best score: ", kn_grid_search.best_score_)

Based on the F1 scores, the Logistic Regression model achieved the highest performance with an average F1 score of 0.6483. It outperformed the Decision Tree Classifier and K-Nearest Neighbors models.

This suggests that the Logistic Regression model is able to predict the presence of sepsis with the highest accuracy.

# Export Key Components
Here is the section to export the important ML objects that will be use to develop an app: Encoder, Scaler, ColumnTransformer, Model, Pipeline, etc.

In [ ]:
#creating a file path to save all the componets in.
if not os.path.exists("key_comp"):
    os.makedirs("key_comp")

In [ ]:
# set the destination path to the "export" directory
destination = os.path.join(".", "key_comp")

In [ ]:
# Get the best LR model
best_lr = lr_grid_search.best_estimator_

In [ ]:
components_dtc = {
    "num_imputer":numerical_imputer,
    "scaler": scaler,
    "models": best_lr 
}

In [ ]:
# Export the LR model
with open('sepssis_model.pkl', 'wb') as f:
    pickle.dump(components_dtc, f)

In [ ]:
!pip list --format=freeze --no-version >key_comp/requirements.txt

# Testing Model

In [ ]:
patient_test.head()

In [ ]:
patient_test.isnull().sum()

In [ ]:
test_data = patient_test.drop(columns=['ID','Insurance'])

In [ ]:
test_num = test_data.columns

In [ ]:
test_attributes = list(test_num)

In [ ]:
numerical_pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')),('scaler', StandardScaler())])

In [ ]:
full_pipeline = ColumnTransformer([('numerical', numerical_pipeline, test_attributes)], remainder='passthrough')

In [ ]:
test_imputed = full_pipeline.fit_transform(X_eval)
test_imputed

In [ ]:
models = pickle.load(open('sepssis_model.pkl','rb'))
models

In [ ]:
model = models['models']

In [ ]:
predictions = model.predict(test_imputed)

In [ ]:
predictions